In [22]:
from typing import List, Type, Literal 
from pydantic import BaseModel
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
import json

# ---------------------------
# 1. Define the Pydantic Schema
# ---------------------------
class ParsedRequest_intent(BaseModel):
    intent: str                 
 

# ---------------------------
# 2. Function to Use ChatOllama with Pydantic Schema
# ---------------------------
def intent(
    llm_model_name: str,
    user_input: str,
    schema: Type[BaseModel]
    
) -> BaseModel:
    system_prompt = """You are a expert in classifying questions into 2 categories. The 2 categories are exact question , needs_clarification. A question is marked as exact if it has concrete details in 3 categories - company name, quarter & Year , metrics to be analysed. It is needs_clarification if the user uses words like analyse, in detail , research, elaborate or if the user doesn't provide specific metrics or company name or year & quarter to be analysed . give the output in JSON format srtictly . JSON has one key and it is called intent . For example ) {"intent": "exact"}"""
    # Load the Ollama model
    llm = ChatOllama(model=llm_model_name)

    # Create and send the prompt
    messages = [
        SystemMessage(content=system_prompt.strip()),
        HumanMessage(content=user_input.strip())
    ]
    response = llm.invoke(messages)

    # Validate using Pydantic
    try:
        raw = response.content.strip()
    
        # Optional: clean triple backticks if LLM returns markdown
        if "```" in raw:
            import re
            match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", raw, re.DOTALL)
            if match:
                raw = match.group(1)
    
        # Step 1: Convert JSON string to Python dict
        parsed_dict = json.loads(raw)
    
        # Step 2: Validate and convert into a Pydantic object
        parsed_model = schema.model_validate(parsed_dict)
    
        # Step 3: Return it (now it has `.model_dump_json()` etc.)
        return parsed_model

    except Exception as e:
        raise ValueError(f"Failed to parse model output: {e}\nRaw Output:\n{response.content}")



        

In [36]:
    
user_input = "Extract the net revenue of citi bank in 2025Q1 "

result = intent(
    llm_model_name="qwen2.5:7b",  # Replace with your loaded Ollama model name
    user_input=user_input,
    schema=ParsedRequest_intent
)

print(result.model_dump_json())


{"intent":"exact"}


In [43]:
def vague (model,question,allowed_banks,allowed_metrics):
    prompt=f"""You are an expert in elaborating and writing explicit questions. Expand the user provided vague question to include more detail along 3 dimensions. Company names , metrics, quarters&Years 
            ---------
            Follow these instructions :
            1. If the user doesn't mention any bank, assume company name as Wells Fargo
            2. If the user doesn't mention quarter or years, assume 1Q2025, 4Q2024, 3Q2024
            3. If the user doesn't provide any metrics  - assume it as Net Income, Earnings per share and total revenue 
            4. Add any additional metrics that you think will be important for senior leadership of a finance company to analyse reports 
            5. Just provide the final output without including the rationale part
            ----------
            Rules:
            - Map any abbreviation or alias to official bank names from this list: {json.dumps(allowed_banks)}
            - Extract all mentioned quarters in "1Q2025" format. Include the previous 2 quarters for each.
            - Extract only metrics listed here: {json.dumps(allowed_metrics)}.
            ----------
            """
    llm=ChatOllama (model=model)
    messages = [
        SystemMessage(content=prompt.strip()),
        HumanMessage(content=question.strip())
    ]
    response = llm.invoke(messages)
    return response
    
allowed_banks = ["JP Morgan Chase", "Bank of America", "Citigroup", "Wells Fargo"]
allowed_metrics = ["EarningsPerShare", "NetIncome", "TotalRevenue", "ReturnOnEquity"]

user_input = "Extract the net revenue of citi bank in 2025Q1"

result = vague(
    model="qwen3:4b",  # Replace with your loaded Ollama model name
    question=user_input,
    allowed_banks=allowed_banks,
    allowed_metrics=allowed_metrics
)
print (result)


        

content='<think>\nOkay, the user wants to extract the net revenue of Citigroup in 2025Q1. Let me start by checking the company name. They mentioned "Citi Bank," which I need to map to the official name. From the list, "Citigroup" is the correct one. Next, the quarters. The user specified 2025Q1, so I should include the previous two quarters: 2024Q4 and 2024Q3. For metrics, they asked for net revenue, which is TotalRevenue. The required metrics are TotalRevenue, EarningsPerShare, NetIncome, and ReturnOnEquity. Since the user only mentioned net revenue, I\'ll include that and add the other three as they are important for analysis. The years are 2025, so the quarters are 2025Q1, 2024Q4, 2024Q3. I need to make sure all the metrics are included as per the rules. Let me structure the final answer with these details.\n</think>\n\nExtract the TotalRevenue, EarningsPerShare, NetIncome, and ReturnOnEquity for Citigroup in 2025Q1, 2024Q4, and 2024Q3.' additional_kwargs={} response_metadata={'mode

In [44]:
from IPython.display import HTML, Markdown 
Markdown(result.content)

<think>
Okay, the user wants to extract the net revenue of Citigroup in 2025Q1. Let me start by checking the company name. They mentioned "Citi Bank," which I need to map to the official name. From the list, "Citigroup" is the correct one. Next, the quarters. The user specified 2025Q1, so I should include the previous two quarters: 2024Q4 and 2024Q3. For metrics, they asked for net revenue, which is TotalRevenue. The required metrics are TotalRevenue, EarningsPerShare, NetIncome, and ReturnOnEquity. Since the user only mentioned net revenue, I'll include that and add the other three as they are important for analysis. The years are 2025, so the quarters are 2025Q1, 2024Q4, 2024Q3. I need to make sure all the metrics are included as per the rules. Let me structure the final answer with these details.
</think>

Extract the TotalRevenue, EarningsPerShare, NetIncome, and ReturnOnEquity for Citigroup in 2025Q1, 2024Q4, and 2024Q3.

In [30]:
from typing import List, Type, Literal 
from pydantic import BaseModel
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
import json

# ---------------------------
# 1. Define the Pydantic Schema
# ---------------------------
class ParsedRequest(BaseModel):
                 
    banks: List[str]            # Must match allowed_banks
    quarters: List[str]         # e.g., "1Q2025", "4Q2024"
    metrics: List[str]          # Must match allowed_metrics


# ---------------------------
# 2. Function to Use ChatOllama with Pydantic Schema
# ---------------------------
def parse_with_chatollama(
        llm_model_name: str,
        user_input: str,
        schema: Type[BaseModel],
        allowed_banks: List[str],
        allowed_metrics: List[str]
    ) -> BaseModel:
    system_prompt = f"""
You are a financial assistant. Your task is to extract structured information from user input and return it in the following JSON format:

{{
  
  "banks": [valid bank names],
  "quarters": ["1Q2025", "4Q2024", "3Q2024"],
  "metrics": [valid metric keys]
}}

Rules:
- Map any abbreviation or alias to official bank names from this list: {json.dumps(allowed_banks)}
- Extract all mentioned quarters in "1Q2025" format. Include the previous 2 quarters for each.
- Extract only metrics listed here: {json.dumps(allowed_metrics)}.
- Output only the JSON structure as shown above, no explanation or markdown.
"""

    # Load the Ollama model
    llm = ChatOllama(model=llm_model_name)

    # Create and send the prompt
    messages = [
        SystemMessage(content=system_prompt.strip()),
        HumanMessage(content=user_input.strip())
    ]
    response = llm.invoke(messages)

    # Validate using Pydantic
    try:
        raw = response.content.strip()
    
        # Optional: clean triple backticks if LLM returns markdown
        if "```" in raw:
            import re
            match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", raw, re.DOTALL)
            if match:
                raw = match.group(1)
    
        # Step 1: Convert JSON string to Python dict
        parsed_dict = json.loads(raw)
    
        # Step 2: Validate and convert into a Pydantic object
        parsed_model = schema.model_validate(parsed_dict)
    
        # Step 3: Return it (now it has `.model_dump_json()` etc.)
        return parsed_model

    except Exception as e:
        raise ValueError(f"Failed to parse model output: {e}\nRaw Output:\n{response.content}")

    

In [35]:

allowed_banks = ["JP Morgan Chase", "Bank of America", "Citigroup", "Wells Fargo"]
allowed_metrics = ["EarningsPerShare", "NetIncome", "TotalRevenue", "ReturnOnEquity"]

user_input = "Extract the net revenue of citi bank in 2025Q1"

result = parse_with_chatollama(
    llm_model_name="qwen2.5:7b",  # Replace with your loaded Ollama model name
    user_input=user_input,
    schema=ParsedRequest,
    allowed_banks=allowed_banks,
    allowed_metrics=allowed_metrics
)

print(result.model_dump_json())


{"banks":["Citigroup"],"quarters":["1Q2025","4Q2024","3Q2024"],"metrics":["NetIncome"]}


In [3]:
import os

dic = r'C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main'
full_path = os.path.join(dic, 'docs')

print(full_path)
print(os.listdir(full_path))
print(os.path.join(full_path,os.listdir(full_path)[0]))
for i in os.listdir(full_path):
    print(os.path.join(full_path,i))


C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main\docs
['boa_ppt_2024_q2.pdf', 'boa_ppt_2024_q3.pdf', 'boa_ppt_2024_q4.pdf', 'boa_ppt_2025_q1.pdf', 'boa_result_2025_q1.pdf', 'citi_result_2024_q1.pdf', 'citi_result_2024_q2.pdf', 'citi_result_2024_q3.pdf', 'citi_result_2024_q4.pdf', 'citi_result_2025_q1.pdf']
C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main\docs\boa_ppt_2024_q2.pdf
C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main\docs\boa_ppt_2024_q2.pdf
C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main\docs\boa_ppt_2024_q3.pdf
C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main\docs\boa_ppt_2024_q4.pdf
C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main\docs\boa_ppt_2025_q1.pdf
C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main\docs\boa_result_2025_q1.pdf
C:\Users\Aksha

In [4]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBox, LTTextLine
import re

def is_natural_language(text):
    # Must contain at least one sentence (simple heuristic)
    return bool(re.search(r"[A-Za-z]{4,}.*\.", text)) and not is_table_like(text)

def is_table_like(text):
    lines = text.strip().splitlines()
    if len(lines) < 2:
        return False

    table_like = 0
    for line in lines:
        tokens = line.strip().split()
        num_tokens = len(tokens)
        numbers = len([t for t in tokens if re.fullmatch(r"[\d,.%$]+", t)])
        symbols = len([t for t in tokens if re.fullmatch(r"[\d,.%$O/(U)-]+", t)])

        if num_tokens >= 3 and numbers / num_tokens > 0.5:
            table_like += 1
        elif len(re.findall(r"\$\s?\d", line)) > 1:  # multiple dollar values
            table_like += 1
        elif len(re.findall(r"\d{2,},", line)) > 1:
            table_like += 1

    return table_like / len(lines) > 0.4

def extract_text_excluding_tables(pdf_path):
    final_text = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, (LTTextBox, LTTextLine)):
                text = element.get_text().strip()
                if text and is_natural_language(text):
                    final_text.append(text)

    return "\n\n".join(final_text).strip()

# for pdf in os.listdir(full_path):
#     print(pdf)
#     text=""
#     text =f"<{pdf}>"
#     clean_text = extract_text_excluding_tables(os.path.join(full_path,pdf))
#     clean_text=text+clean_text
#     text =f"</{pdf}"
#     text=clean_text+text
#     #print(text)


In [5]:
def parse_filename_metadata(filename: str):
    name = os.path.splitext(os.path.basename(filename))[0]
    parts = name.split("_")
    bank_map = {
        "jpm": "JP Morgan Chase",
        "boa": "Bank of America",
        "citi": "Citigroup",
        "gs": "Goldman Sachs",
        "ms": "Morgan Stanley",
    }
    bank_code = parts[0].lower()
    quarter = parts[1].upper() if len(parts) > 1 else "UNKNOWN"
    bank = bank_map.get(bank_code, bank_code.upper())
    return bank, quarter

In [6]:
class ParsedRequest(BaseModel):
    intent: str
    banks: List[str]
    quarters: List[str]
    metrics: List[str]


In [7]:
def create_chunks_with_ollama(text: str, metadata: dict = None):
    embedder = OllamaEmbeddings(model="nomic-embed-text")
    chunker = SemanticChunker(embeddings=embedder, min_chunk_size=2000)

    doc = Document(page_content=text, metadata=metadata or {})
    return chunker.split_documents([doc])


In [8]:
def search_chunks(chunks, parsed_query: ParsedRequest, top_k=5):
    embedder = OllamaEmbeddings(model="nomic-embed-text")
    vectorstore = FAISS.from_documents(chunks, embedder)

    query_text = (
        f"Find information about {', '.join(parsed_query.metrics)} "
        f"for banks like {', '.join(parsed_query.banks)} "
        f"during quarters such as {', '.join(parsed_query.quarters)}"
    )

    return vectorstore.similarity_search(query_text, k=top_k)


In [9]:
def rerank_with_chatollama(chunks, parsed_query: ParsedRequest):
    llm = ChatOllama(model="llama3.2:3b")
    context = "\n\n".join([chunk.page_content for chunk in chunks])

    prompt = (
        f"You are a financial analyst assistant.\n\n"
        f"Query:\n{parsed_query.model_dump_json(indent=2)}\n\n"
        f"Extracted text:\n{context}\n\n"
        f"Please summarize the relevant details in a table or paragraph based on the query intent."
    )

    response = llm([HumanMessage(content=prompt)])
    return response.content


In [10]:
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBox, LTTextLine
import re
import json
from typing import List
from pydantic import BaseModel
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage
import os
import re
from glob import glob
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBox, LTTextLine
full_path = r"C:\Users\Akshaya V\git\Earnings research\Earnings_agent\PublicReportResearch-main\docs"

all_chunks = []

for pdf in os.listdir(full_path):
    if not pdf.lower().endswith(".pdf"):
        continue

    pdf_path = os.path.join(full_path, pdf)
    print(f"📄 Processing: {pdf}")

    bank, quarter = parse_filename_metadata(pdf)
    clean_text = extract_text_excluding_tables(pdf_path)

    # Wrap in custom tags for traceability
    tagged_text = f"<{bank} {quarter}>\n{clean_text}\n</{bank} {quarter}>"

    # Metadata
    metadata = {
        "source": pdf,
        "bank": bank,
        "quarter": quarter
    }

    # Chunk with metadata
    chunks = create_chunks_with_ollama(tagged_text, metadata)
    all_chunks.extend(chunks)

# ✅ Print preview
print(f"\nTotal Chunks Created: {len(all_chunks)}")
for i, chunk in enumerate(all_chunks[:3]):
    print(f"\n--- Chunk {i+1} ---")
    print("Metadata:", chunk.metadata)
    print("Content preview:", chunk.page_content[:300], "...\n")

📄 Processing: boa_ppt_2024_q2.pdf
📄 Processing: boa_ppt_2024_q3.pdf
📄 Processing: boa_ppt_2024_q4.pdf
📄 Processing: boa_ppt_2025_q1.pdf
📄 Processing: boa_result_2025_q1.pdf
📄 Processing: citi_result_2024_q1.pdf


Cannot set gray non-stroke color because /'P43' is an invalid float value
Cannot set gray non-stroke color because /'P45' is an invalid float value
Cannot set gray non-stroke color because /'P46' is an invalid float value
Cannot set gray non-stroke color because /'P61' is an invalid float value
Cannot set gray non-stroke color because /'P62' is an invalid float value
Cannot set gray non-stroke color because /'P66' is an invalid float value
Cannot set gray non-stroke color because /'P67' is an invalid float value
Cannot set gray non-stroke color because /'P71' is an invalid float value
Cannot set gray non-stroke color because /'P72' is an invalid float value
Cannot set gray non-stroke color because /'P78' is an invalid float value
Cannot set gray non-stroke color because /'P79' is an invalid float value
Cannot set gray non-stroke color because /'P83' is an invalid float value
Cannot set gray non-stroke color because /'P84' is an invalid float value
Cannot set gray non-stroke color becau

📄 Processing: citi_result_2024_q2.pdf


Cannot set gray non-stroke color because /'P39' is an invalid float value
Cannot set gray non-stroke color because /'P41' is an invalid float value
Cannot set gray non-stroke color because /'P42' is an invalid float value
Cannot set gray non-stroke color because /'P56' is an invalid float value
Cannot set gray non-stroke color because /'P57' is an invalid float value
Cannot set gray non-stroke color because /'P63' is an invalid float value
Cannot set gray non-stroke color because /'P64' is an invalid float value
Cannot set gray non-stroke color because /'P68' is an invalid float value
Cannot set gray non-stroke color because /'P69' is an invalid float value
Cannot set gray non-stroke color because /'P75' is an invalid float value
Cannot set gray non-stroke color because /'P76' is an invalid float value
Cannot set gray non-stroke color because /'P80' is an invalid float value
Cannot set gray non-stroke color because /'P81' is an invalid float value
Cannot set gray non-stroke color becau

📄 Processing: citi_result_2024_q3.pdf


...

📄 Processing: citi_result_2024_q4.pdf


Cannot set gray non-stroke color because /'P45' is an invalid float value
Cannot set gray non-stroke color because /'P47' is an invalid float value
Cannot set gray non-stroke color because /'P48' is an invalid float value
Cannot set gray non-stroke color because /'P49' is an invalid float value
Cannot set gray non-stroke color because /'P53' is an invalid float value
Cannot set gray non-stroke color because /'P54' is an invalid float value
Cannot set gray non-stroke color because /'P55' is an invalid float value
Cannot set gray non-stroke color because /'P60' is an invalid float value
Cannot set gray non-stroke color because /'P61' is an invalid float value
Cannot set gray non-stroke color because /'P62' is an invalid float value
Cannot set gray non-stroke color because /'P67' is an invalid float value
Cannot set gray non-stroke color because /'P68' is an invalid float value
Cannot set gray non-stroke color because /'P70' is an invalid float value
Cannot set gray non-stroke color becau

📄 Processing: citi_result_2025_q1.pdf


Cannot set gray non-stroke color because /'P40' is an invalid float value
Cannot set gray non-stroke color because /'P42' is an invalid float value
Cannot set gray non-stroke color because /'P43' is an invalid float value
Cannot set gray non-stroke color because /'P49' is an invalid float value
Cannot set gray non-stroke color because /'P50' is an invalid float value
Cannot set gray non-stroke color because /'P54' is an invalid float value
Cannot set gray non-stroke color because /'P55' is an invalid float value
Cannot set gray non-stroke color because /'P59' is an invalid float value
Cannot set gray non-stroke color because /'P60' is an invalid float value
Cannot set gray non-stroke color because /'P66' is an invalid float value
Cannot set gray non-stroke color because /'P67' is an invalid float value
Cannot set gray non-stroke color because /'P73' is an invalid float value
Cannot set gray non-stroke color because /'P74' is an invalid float value
Cannot set gray non-stroke color becau


Total Chunks Created: 99

--- Chunk 1 ---
Metadata: {'source': 'boa_ppt_2024_q2.pdf', 'bank': 'Bank of America', 'quarter': 'PPT'}
Content preview: <Bank of America PPT>
Lee McEntire 
Good morning. Welcome. Thank you for joining the call to review our second quarter results. Our earnings 
release documents are available on the Investor Relations section of the bankofamerica.com website, and 
they include the earnings presentation that we will m ...


--- Chunk 2 ---
Metadata: {'source': 'boa_ppt_2024_q2.pdf', 'bank': 'Bank of America', 'quarter': 'PPT'}
Content preview: We're not complacent with the 
success you see on this page. We continue to strategically invest in our core businesses. A few examples. While we have the leading retail deposit share in America, we continue to invest and have 
opened 11 new financial centers this quarter in the first half of the ye ...


--- Chunk 3 ---
Metadata: {'source': 'boa_ppt_2024_q2.pdf', 'bank': 'Bank of America', 'quarter': 'PPT'}
Content pr

In [12]:
structured_query = '''{
    "intent": "in-depth",
    "banks": ["JP Morgan Chase", "Bank of America"],
    "quarters": ["1Q2025", "4Q2024", "3Q2024"],
    "metrics": ["EarningsPerShare", "NetIncome"]
}'''

parsed_query = ParsedRequest.model_validate(json.loads(structured_query))

# Search top relevant chunks based on user's intent
matched_chunks = search_chunks(all_chunks, parsed_query, top_k=5)

# Generate final answer with LLM (LLaMA 3.2)
final_answer = rerank_with_chatollama(matched_chunks, parsed_query)
print(final_answer)

Here is a summary of the relevant details in a paragraph:

Bank of America reported its quarterly earnings, with net income of $1.9 billion and revenue of $6.0 billion. The company saw an increase in non-interest expense by 6% driven by investments in technology and operations. However, the revenue was flat due to gains from leveraged finance positions offsetting lower Net Interest Income (NII). Bank of America also reported its market share and relationships with major corporations, including 78% coverage of the Global Fortune 500 and 95% coverage of the U.S. corporate banking market.

Additionally, the company announced plans to expand its network in key markets, focusing on building out branches in cities like Columbus, where it already has a significant presence. This will allow for more efficient operations and better customer engagement, particularly in terms of digital services.


In [16]:
structured_query = result.model_dump_json()

parsed_query = ParsedRequest.model_validate(json.loads(structured_query))

# Search top relevant chunks based on user's intent
matched_chunks = search_chunks(all_chunks, parsed_query, top_k=5)

# Generate final answer with LLM (LLaMA 3.2)
final_answer = rerank_with_chatollama(matched_chunks, parsed_query)
print(final_answer)

Based on the provided text, here is a summary of the relevant details:

**Bank Performance**

* Bank of America (BofA) reported $6.0 billion in revenue, net of interest expense, for 2Q2025.
* Net income was $1.9 billion.
* The bank's efficiency ratio improved due to increased operating leverage from NII growth.

**Business Highlights**

* BofA maintained its position as the number one investment banking firm and won the "U.S. Corporate Banking & Best Bank Award".
* The bank has relationships with 78% of the Global Fortune 500 and 95% of the U.S. Global Fortune 500.
* BofA's average deposits were $100 million per branch, exceeding expectations due to density and capacity.

**Future Plans**

* The bank aims to expand its presence in top markets across the country to cover the American population efficiently and effectively.
* BofA is focusing on building a network in key locations, such as Columbus, Ohio, rather than expanding rapidly.
* The bank expects to continue growing organically a

In [45]:
import requests
import pandas as pd
import json
import time
from datetime import datetime, timedelta
import os
from typing import Dict, List, Optional
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import re
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class SECBankDataExtractor:
    def __init__(self, user_agent: str = "akshayavaradu@gmail.com"):
        """
        Initialize the SEC data extractor
        
        Args:
            user_agent: Required user agent for SEC API requests
        """
        self.user_agent = user_agent
        self.base_url = "https://data.sec.gov"
        self.headers = {
            'User-Agent': self.user_agent,
            'Accept-Encoding': 'gzip, deflate',
            'Host': 'data.sec.gov'
        }
        
        # Top 20 US Banks by Assets (CIK numbers)
        self.top_banks = {
            'JPMorgan Chase & Co.': '0000019617',
            'Bank of America Corp': '0000070858'
            
        }
        
        # Top 50 financial metrics prioritized by importance for banking stakeholders
        # Order: Most critical metrics first (profitability, capital, risk) -> operational -> growth -> regulatory
        self.target_metrics = [
            # TIER 1: Core Profitability & Performance (Most Important)
            'NetIncome',                           # Bottom line profitability
            'TotalRevenue',                        # Top line revenue
            'NetInterestIncome',                   # Core banking income
            'ReturnOnEquity',                      # Key profitability ratio
            'ReturnOnAssets',                      # Asset efficiency
            'EarningsPerShare',                    # Shareholder value
            'EarningsPerShareDiluted',             # Diluted EPS
            
            # TIER 2: Capital & Balance Sheet Strength
            'TotalAssets',                         # Bank size/scale
            'ShareholdersEquity',                  # Capital base
            'TotalDeposits',                       # Funding base
            'TotalLoans',                          # Core asset
            'BookValuePerShare',                   # Per-share equity value
            'TangibleBookValuePerShare',           # Tangible equity value
            'Tier1CapitalRatio',                   # Regulatory capital
            
            # TIER 3: Risk Management & Credit Quality
            'ProvisionForLoanLosses',              # Credit risk expense
            'AllowanceForLoanLosses',              # Credit loss reserves
            'NonPerformingLoans',                  # Problem assets
            'ChargeOffs',                          # Realized losses
            'LoanLossReserveRatio',                # Reserve coverage
            'NonPerformingAssetRatio',             # Asset quality
            
            # TIER 4: Income Statement Detail
            'InterestIncome',                      # Interest revenue
            'InterestExpense',                     # Interest costs
            'NonInterestIncome',                   # Fee income
            'NonInterestExpense',                  # Operating expenses
            'OperatingExpenses',                   # Total opex
            'PersonnelExpense',                    # Staff costs
            'OccupancyExpense',                    # Facility costs
            
            # TIER 5: Operational Efficiency
            'EfficiencyRatio',                     # Cost efficiency
            'NetInterestMargin',                   # Spread profitability
            'CostOfFunds',                         # Funding cost
            'AssetTurnover',                       # Asset utilization
            'OperatingLeverage',                   # Operating efficiency
            
            # TIER 6: Growth & Market Metrics
            'TotalRevenueGrowth',                  # Revenue growth
            'LoanGrowth',                          # Loan portfolio growth
            'DepositGrowth',                       # Deposit growth
            'TangibleEquityRatio',                 # Tangible capital ratio
            'LeverageRatio',                       # Financial leverage
            
            # TIER 7: Trading & Investment Banking
            'TradingRevenue',                      # Trading income
            'InvestmentBankingRevenue',            # IB fees
            'TrustAndInvestmentFees',              # Wealth management
            'ServiceCharges',                      # Service fees
            'CardRevenue',                         # Credit card income
            
            # TIER 8: Regulatory & Capital Management
            'CommonEquityTier1Ratio',              # CET1 ratio
            'TotalCapitalRatio',                   # Total capital
            'RiskWeightedAssets',                  # RWA
            'LeverageCapitalRatio',                # Leverage ratio
            'LiquidityRatio',                      # Liquidity position
            
            # TIER 9: Additional Balance Sheet Items
            'TradingAssets',                       # Trading portfolio
            'AvailableForSaleSecurities',          # AFS securities
            'HeldToMaturitySecurities',            # HTM securities
            'Goodwill',                            # Goodwill asset
            'IntangibleAssets'                     # Other intangibles
        ]
    
    def get_company_facts(self, cik: str) -> Optional[Dict]:
        """
        Get company facts from SEC API
        
        Args:
            cik: Company CIK number
            
        Returns:
            Dictionary containing company facts or None if error
        """
        try:
            url = f"{self.base_url}/api/xbrl/companyfacts/CIK{cik.zfill(10)}.json"
            response = requests.get(url, headers=self.headers)
            
            if response.status_code == 200:
                return response.json()
            else:
                logger.warning(f"Failed to get company facts for CIK {cik}: {response.status_code}")
                return None
                
        except Exception as e:
            logger.error(f"Error getting company facts for CIK {cik}: {str(e)}")
            return None
    
    def get_company_filings(self, cik: str, start_date: str, end_date: str) -> Optional[Dict]:
        """
        Get company filings from SEC API
        
        Args:
            cik: Company CIK number
            start_date: Start date in YYYY-MM-DD format
            end_date: End date in YYYY-MM-DD format
            
        Returns:
            Dictionary containing filings data or None if error
        """
        try:
            url = f"{self.base_url}/api/xbrl/submissions/CIK{cik.zfill(10)}.json"
            response = requests.get(url, headers=self.headers)
            
            if response.status_code == 200:
                data = response.json()
                # Filter filings by date and form type
                filings = []
                recent_filings = data.get('filings', {}).get('recent', {})
                
                if recent_filings:
                    for i in range(len(recent_filings.get('form', []))):
                        form_type = recent_filings['form'][i]
                        filing_date = recent_filings['filingDate'][i]
                        
                        if form_type in ['10-K', '10-Q'] and start_date <= filing_date <= end_date:
                            filings.append({
                                'form': form_type,
                                'filingDate': filing_date,
                                'accessionNumber': recent_filings['accessionNumber'][i],
                                'reportDate': recent_filings.get('reportDate', [''])[i],
                                'primaryDocument': recent_filings.get('primaryDocument', [''])[i]
                            })
                
                return {'filings': filings}
            else:
                logger.warning(f"Failed to get filings for CIK {cik}: {response.status_code}")
                return None
                
        except Exception as e:
            logger.error(f"Error getting filings for CIK {cik}: {str(e)}")
            return None
    
    def extract_financial_metrics(self, company_facts: Dict, start_date: str, end_date: str) -> List[Dict]:
        """
        Extract financial metrics from company facts
        
        Args:
            company_facts: Company facts data from SEC API
            start_date: Start date in YYYY-MM-DD format
            end_date: End date in YYYY-MM-DD format
            
        Returns:
            List of dictionaries containing extracted metrics
        """
        metrics_data = []
        
        try:
            facts = company_facts.get('facts', {})
            us_gaap = facts.get('us-gaap', {})
            dei = facts.get('dei', {})
            
            # Comprehensive GAAP tags mapping to our 50 target metrics
            gaap_mapping = {
                # TIER 1: Core Profitability & Performance
                'NetIncome': ['NetIncomeLoss', 'ProfitLoss', 'NetIncomeLossAvailableToCommonStockholdersBasic', 'IncomeLossFromContinuingOperations'],
                'TotalRevenue': ['Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax', 'InterestAndDividendIncomeOperating', 'TotalRevenues'],
                'NetInterestIncome': ['InterestIncomeExpenseNet', 'NetInterestIncome', 'InterestIncomeExpenseAfterProvisionForLoanLoss'],
                'ReturnOnEquity': ['ReturnOnAverageEquity', 'ReturnOnEquity'],
                'ReturnOnAssets': ['ReturnOnAverageAssets', 'ReturnOnAssets'],
                'EarningsPerShare': ['EarningsPerShareBasic', 'IncomeLossFromContinuingOperationsPerBasicShare'],
                'EarningsPerShareDiluted': ['EarningsPerShareDiluted', 'IncomeLossFromContinuingOperationsPerDilutedShare'],
                
                # TIER 2: Capital & Balance Sheet Strength
                'TotalAssets': ['Assets', 'AssetsCurrent', 'AssetsNoncurrent'],
                'ShareholdersEquity': ['StockholdersEquity', 'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest', 'EquityAttributableToParent'],
                'TotalDeposits': ['Deposits', 'DepositsTotal', 'InterestBearingDeposits', 'NoninterestBearingDeposits'],
                'TotalLoans': ['LoansAndLeasesReceivableNetOfAllowance', 'LoansAndLeasesReceivableGross', 'LoansReceivableNet'],
                'BookValuePerShare': ['BookValuePerShare', 'StockholdersEquityPerShare'],
                'TangibleBookValuePerShare': ['TangibleBookValuePerShare'],
                'Tier1CapitalRatio': ['Tier1CapitalRatio', 'CapitalAdequacyTier1CapitalRatio'],
                
                # TIER 3: Risk Management & Credit Quality
                'ProvisionForLoanLosses': ['ProvisionForLoanAndLeaseLosses', 'ProvisionForCreditLosses', 'ProvisionForDoubtfulAccounts'],
                'AllowanceForLoanLosses': ['AllowanceForLoanAndLeaseLosses', 'AllowanceForCreditLossesFinancingReceivables'],
                'NonPerformingLoans': ['LoansAndLeasesReceivableNonaccrual', 'NonperformingLoans'],
                'ChargeOffs': ['LoansAndLeasesReceivableChargeOffs', 'ChargeOffsLoansAndLeases'],
                'LoanLossReserveRatio': ['LoanLossReserveRatio'],
                'NonPerformingAssetRatio': ['NonperformingAssetRatio'],
                
                # TIER 4: Income Statement Detail
                'InterestIncome': ['InterestAndFeeIncomeLoansAndLeases', 'InterestIncomeOperating', 'InterestAndDividendIncomeOperating', 'InterestIncomeLoansAndLeases'],
                'InterestExpense': ['InterestExpense', 'InterestExpenseDeposits', 'InterestExpenseDebt', 'InterestExpenseBorrowings'],
                'NonInterestIncome': ['NoninterestIncome', 'RevenuesExcludingInterestAndDividends', 'FeesAndCommissions'],
                'NonInterestExpense': ['NoninterestExpense', 'OperatingExpenses', 'GeneralAndAdministrativeExpense'],
                'OperatingExpenses': ['OperatingExpenses', 'CostsAndExpenses', 'OperatingCostsAndExpenses'],
                'PersonnelExpense': ['LaborAndRelatedExpense', 'EmployeeRelatedExpense', 'SalariesAndWages'],
                'OccupancyExpense': ['OccupancyNet', 'OccupancyAndEquipmentExpense'],
                
                # TIER 5: Operational Efficiency
                'EfficiencyRatio': ['EfficiencyRatio'],
                'NetInterestMargin': ['NetInterestMargin'],
                'CostOfFunds': ['CostOfFunds'],
                'AssetTurnover': ['AssetTurnover'],
                'OperatingLeverage': ['OperatingLeverage'],
                
                # TIER 6: Growth & Market Metrics
                'TotalRevenueGrowth': ['RevenueGrowthRate'],
                'LoanGrowth': ['LoanGrowthRate'],
                'DepositGrowth': ['DepositGrowthRate'],
                'TangibleEquityRatio': ['TangibleEquityRatio'],
                'LeverageRatio': ['LeverageRatio', 'DebtToEquityRatio'],
                
                # TIER 7: Trading & Investment Banking
                'TradingRevenue': ['TradingGainsLosses', 'TradingAccountProfitLoss', 'SecuritiesGainLoss'],
                'InvestmentBankingRevenue': ['InvestmentBankingRevenue', 'UnderwritingIncome'],
                'TrustAndInvestmentFees': ['TrustFeesRevenue', 'InvestmentManagementAndTrustFees'],
                'ServiceCharges': ['ServiceChargesOnDepositAccounts', 'ServiceCharges'],
                'CardRevenue': ['CreditCardIncome', 'CreditCardFees'],
                
                # TIER 8: Regulatory & Capital Management
                'CommonEquityTier1Ratio': ['CommonEquityTier1CapitalRatio', 'Tier1CommonCapitalRatio'],
                'TotalCapitalRatio': ['TotalCapitalRatio', 'CapitalAdequacyTotalCapitalRatio'],
                'RiskWeightedAssets': ['RiskWeightedAssets'],
                'LeverageCapitalRatio': ['LeverageRatio', 'CapitalAdequacyLeverageRatio'],
                'LiquidityRatio': ['LiquidityRatio', 'LiquidityCoverageRatio'],
                
                # TIER 9: Additional Balance Sheet Items
                'TradingAssets': ['TradingSecuritiesDebt', 'TradingSecuritiesEquity', 'TradingSecurities'],
                'AvailableForSaleSecurities': ['AvailableForSaleSecuritiesDebtSecurities', 'MarketableSecuritiesAvailableForSale'],
                'HeldToMaturitySecurities': ['HeldToMaturitySecurities', 'DebtSecuritiesHeldToMaturity'],
                'Goodwill': ['Goodwill'],
                'IntangibleAssets': ['IntangibleAssetsNetExcludingGoodwill', 'FiniteLivedIntangibleAssetsNet']
            }
            
            # Get all available periods
            all_periods = set()
            for metric_group in gaap_mapping.values():
                for tag in metric_group:
                    if tag in us_gaap:
                        for unit_type in us_gaap[tag].get('units', {}):
                            for entry in us_gaap[tag]['units'][unit_type]:
                                if 'end' in entry and start_date <= entry['end'] <= end_date:
                                    all_periods.add(entry['end'])
            
            # Extract data for each period
            for period_end in sorted(all_periods):
                period_data = {
                    'Datetime': period_end,
                    'CompanyName': company_facts.get('entityName', 'Unknown')
                }
                
                # Extract each metric
                for target_metric, possible_tags in gaap_mapping.items():
                    value = None
                    for tag in possible_tags:
                        if tag in us_gaap:
                            for unit_type in us_gaap[tag].get('units', {}):
                                for entry in us_gaap[tag]['units'][unit_type]:
                                    if entry.get('end') == period_end and 'val' in entry:
                                        # Prefer quarterly data (form 10-Q) over annual, and recent over old
                                        if entry.get('form') in ['10-Q', '10-K']:
                                            if value is None or entry.get('form') == '10-Q':
                                                value = entry['val']
                                            break
                                if value is not None:
                                    break
                        if value is not None:
                            break
                    
                    period_data[target_metric] = value
                
                # Only add if we have some meaningful data (at least core metrics)
                core_metrics = ['NetIncome', 'TotalRevenue', 'TotalAssets', 'ShareholdersEquity']
                if any(period_data.get(metric) is not None for metric in core_metrics):
                    metrics_data.append(period_data)
        
        except Exception as e:
            logger.error(f"Error extracting metrics: {str(e)}")
        
        return metrics_data
    
    def calculate_derived_metrics(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate derived financial metrics that can't be directly extracted
        
        Args:
            df: DataFrame with base financial metrics
            
        Returns:
            DataFrame with additional calculated metrics
        """
        try:
            # Calculate ROE if not available (Net Income / Shareholders Equity)
            mask = (df['ReturnOnEquity'].isna()) & (df['NetIncome'].notna()) & (df['ShareholdersEquity'].notna()) & (df['ShareholdersEquity'] != 0)
            df.loc[mask, 'ReturnOnEquity'] = (df.loc[mask, 'NetIncome'] / df.loc[mask, 'ShareholdersEquity']) * 100
            
            # Calculate ROA if not available (Net Income / Total Assets)
            mask = (df['ReturnOnAssets'].isna()) & (df['NetIncome'].notna()) & (df['TotalAssets'].notna()) & (df['TotalAssets'] != 0)
            df.loc[mask, 'ReturnOnAssets'] = (df.loc[mask, 'NetIncome'] / df.loc[mask, 'TotalAssets']) * 100
            
            # Calculate Book Value Per Share if not available
            mask = (df['BookValuePerShare'].isna()) & (df['ShareholdersEquity'].notna())
            # Note: Would need shares outstanding data which may not be readily available
            
            # Calculate Efficiency Ratio if not available (Non-Interest Expense / (Net Interest Income + Non-Interest Income))
            mask = (df['EfficiencyRatio'].isna()) & (df['NonInterestExpense'].notna()) & \
                   ((df['NetInterestIncome'].notna()) | (df['NonInterestIncome'].notna()))
            
            revenue_base = df['NetInterestIncome'].fillna(0) + df['NonInterestIncome'].fillna(0)
            df.loc[mask & (revenue_base != 0), 'EfficiencyRatio'] = (df.loc[mask & (revenue_base != 0), 'NonInterestExpense'] / revenue_base.loc[mask & (revenue_base != 0)]) * 100
            
            # Calculate Net Interest Margin if not available
            # This would require average earning assets data which may not be available
            
            logger.info("Calculated derived financial metrics")
            
        except Exception as e:
            logger.error(f"Error calculating derived metrics: {str(e)}")
        
        return df
    
    def download_bank_data(self, start_date: str = "2019-01-01", end_date: str = "2025-12-31") -> pd.DataFrame:
        """
        Download earnings data for all banks
        
        Args:
            start_date: Start date in YYYY-MM-DD format
            end_date: End date in YYYY-MM-DD format
            
        Returns:
            DataFrame containing all bank earnings data
        """
        all_data = []
        
        logger.info(f"Starting data download for {len(self.top_banks)} banks from {start_date} to {end_date}")
        logger.info(f"Extracting {len(self.target_metrics)} financial metrics")
        
        for i, (bank_name, cik) in enumerate(self.top_banks.items()):
            logger.info(f"Processing {i+1}/{len(self.top_banks)}: {bank_name}")
            
            try:
                # Get company facts
                company_facts = self.get_company_facts(cik)
                if company_facts:
                    # Extract metrics
                    bank_metrics = self.extract_financial_metrics(company_facts, start_date, end_date)
                    all_data.extend(bank_metrics)
                    
                    logger.info(f"Extracted {len(bank_metrics)} records for {bank_name}")
                else:
                    logger.warning(f"No company facts found for {bank_name}")
                
                # Rate limiting - SEC allows 10 requests per second
                time.sleep(0.1)
                
            except Exception as e:
                logger.error(f"Error processing {bank_name}: {str(e)}")
                continue
        
        # Create DataFrame
        df = pd.DataFrame(all_data)
        
        if not df.empty:
            # Convert datetime column
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            
            # Ensure all target metrics are present as columns (even if empty)
            for metric in self.target_metrics:
                if metric not in df.columns:
                    df[metric] = None
            
            # Reorder columns: Datetime, CompanyName, then metrics in priority order
            column_order = ['Datetime', 'CompanyName'] + self.target_metrics
            df = df.reindex(columns=column_order)
            
            # Calculate derived metrics
            df = self.calculate_derived_metrics(df)
            
            # Sort by company and date
            df = df.sort_values(['CompanyName', 'Datetime'])
            
            # Reset index
            df = df.reset_index(drop=True)
            
            logger.info(f"Successfully extracted {len(df)} total records with {len(self.target_metrics)} metrics")
        else:
            logger.warning("No data was extracted")
        
        return df
    
    def save_to_excel(self, df: pd.DataFrame, filename: str = "bank_earnings_data.xlsx"):
        """
        Save DataFrame to Excel file with enhanced formatting
        
        Args:
            df: DataFrame to save
            filename: Output filename
        """
        try:
            # Create Excel writer
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                # Main data sheet
                df.to_excel(writer, sheet_name='Bank_Earnings_Data', index=False)
                
                # Summary sheet
                if not df.empty:
                    summary_data = []
                    for company in df['CompanyName'].unique():
                        company_data = df[df['CompanyName'] == company]
                        latest_data = company_data.iloc[-1] if len(company_data) > 0 else None
                        
                        summary_data.append({
                            'Company': company,
                            'Records': len(company_data),
                            'Date_Range': f"{company_data['Datetime'].min().strftime('%Y-%m-%d')} to {company_data['Datetime'].max().strftime('%Y-%m-%d')}",
                            'Latest_Total_Assets': latest_data['TotalAssets'] if latest_data is not None else None,
                            'Latest_Net_Income': latest_data['NetIncome'] if latest_data is not None else None,
                            'Latest_ROE': latest_data['ReturnOnEquity'] if latest_data is not None else None,
                            'Latest_ROA': latest_data['ReturnOnAssets'] if latest_data is not None else None,
                            'Data_Quality_Score': f"{(company_data[self.target_metrics].notna().sum().sum() / (len(company_data) * len(self.target_metrics)) * 100):.1f}%"
                        })
                    
                    summary_df = pd.DataFrame(summary_data)
                    summary_df.to_excel(writer, sheet_name='Summary', index=False)
                
                # Metrics Dictionary sheet
                metrics_info = []
                tier_descriptions = {
                    'TIER 1': 'Core Profitability & Performance - Most critical metrics for stakeholder decision making',
                    'TIER 2': 'Capital & Balance Sheet Strength - Financial stability and scale indicators',
                    'TIER 3': 'Risk Management & Credit Quality - Credit risk and asset quality measures',
                    'TIER 4': 'Income Statement Detail - Detailed revenue and expense components',
                    'TIER 5': 'Operational Efficiency - Cost management and operational performance',
                    'TIER 6': 'Growth & Market Metrics - Growth rates and market position indicators',
                    'TIER 7': 'Trading & Investment Banking - Specialized revenue streams',
                    'TIER 8': 'Regulatory & Capital Management - Regulatory compliance metrics',
                    'TIER 9': 'Additional Balance Sheet Items - Supporting balance sheet components'
                }
                
                current_tier = None
                tier_counters = {'TIER 1': 0, 'TIER 2': 0, 'TIER 3': 0, 'TIER 4': 0, 'TIER 5': 0, 
                               'TIER 6': 0, 'TIER 7': 0, 'TIER 8': 0, 'TIER 9': 0}
                
                for i, metric in enumerate(self.target_metrics):
                    # Determine tier based on position
                    if i < 7: tier = 'TIER 1'
                    elif i < 15: tier = 'TIER 2'
                    elif i < 21: tier = 'TIER 3'
                    elif i < 28: tier = 'TIER 4'
                    elif i < 33: tier = 'TIER 5'
                    elif i < 38: tier = 'TIER 6'
                    elif i < 43: tier = 'TIER 7'
                    elif i < 48: tier = 'TIER 8'
                    else: tier = 'TIER 9'
                    
                    tier_counters[tier] += 1
                    
                    metrics_info.append({
                        'Metric_Name': metric,
                        'Priority_Rank': i + 1,
                        'Tier': tier,
                        'Tier_Description': tier_descriptions[tier],
                        'Column_Position': i + 3  # +3 for Datetime and CompanyName columns
                    })
                
                metrics_df = pd.DataFrame(metrics_info)
                metrics_df.to_excel(writer, sheet_name='Metrics_Dictionary', index=False)
            
            logger.info(f"Data saved to {filename}")
            logger.info(f"Excel file contains 3 sheets: Bank_Earnings_Data, Summary, and Metrics_Dictionary")
            
        except Exception as e:
            logger.error(f"Error saving to Excel: {str(e)}")

def main():
    """
    Main function to run the SEC bank data extraction
    """
    # Initialize extractor
    # IMPORTANT: Replace with your actual contact information
    extractor = SECBankDataExtractor(user_agent="YourCompany yourname@yourcompany.com")
    
    # Set date range (default 2019-2025)
    start_date = "2019-01-01"
    end_date = "2025-12-31"
    
    print(f"SEC Bank Earnings Data Extractor - Enhanced Version")
    print(f"==================================================")
    print(f"Downloading data from {start_date} to {end_date}")
    print(f"Target banks: {len(extractor.top_banks)}")
    print(f"Target metrics: {len(extractor.target_metrics)} (prioritized by stakeholder importance)")
    print()
    
    # Display metric tiers
    print("Metric Priority Tiers:")
    print("TIER 1 (1-7): Core Profitability & Performance")
    print("TIER 2 (8-15): Capital & Balance Sheet Strength") 
    print("TIER 3 (16-21): Risk Management & Credit Quality")
    print("TIER 4 (22-28): Income Statement Detail")
    print("TIER 5 (29-33): Operational Efficiency")
    print("TIER 6 (34-38): Growth & Market Metrics")
    print("TIER 7 (39-43): Trading & Investment Banking")
    print("TIER 8 (44-48): Regulatory & Capital Management")
    print("TIER 9 (49-50): Additional Balance Sheet Items")
    print()
    
    # Download data
    df = extractor.download_bank_data(start_date, end_date)
    
    if not df.empty:
        print(f"\nData extraction completed!")
        print(f"Total records: {len(df)}")
        print(f"Companies with data: {df['CompanyName'].nunique()}")
        print(f"Date range: {df['Datetime'].min()} to {df['Datetime'].max()}")
        
        # Save to Excel
        output_filename = f"bank_earnings_data_{start_date}_{end_date}.xlsx"
        extractor.save_to_excel(df, output_filename)
        
        print(f"\nData saved to: {output_filename}")
        
        # Display data quality metrics
        total_possible_values = len(df) * len(extractor.target_metrics)
        actual_values = df[extractor.target_metrics].notna().sum().sum()
        data_completeness = (actual_values / total_possible_values) * 100
        
        print(f"\nData Quality Summary:")
        print(f"Data completeness: {data_completeness:.1f}%")
        print(f"Total possible data points: {total_possible_values:,}")
        print(f"Actual data points extracted: {actual_values:,}")
        
        # Show top metrics by data availability
        print(f"\nTop 10 metrics by data availability:")
        metric_availability = df[extractor.target_metrics].notna().sum().sort_values(ascending=False)
        for i, (metric, count) in enumerate(metric_availability.head(10).items()):
            percentage = (count / len(df)) * 100
            print(f"{i+1:2d}. {metric:<25} {count:4d} records ({percentage:5.1f}%)")
        
        # Display sample data for most important metrics
        important_columns = ['Datetime', 'CompanyName'] + extractor.target_metrics[:10]
        print(f"\nSample data preview (Top 10 priority metrics):")
        print(df[important_columns].head(3).to_string(index=False))
        
        # Show metrics by tier
        print(f"\nMetrics organized by priority tiers:")
        tier_ranges = [
            (1, 7, "TIER 1: Core Profitability & Performance"),
            (8, 15, "TIER 2: Capital & Balance Sheet Strength"),
            (16, 21, "TIER 3: Risk Management & Credit Quality"),
            (22, 28, "TIER 4: Income Statement Detail"),
            (29, 33, "TIER 5: Operational Efficiency"),
            (34, 38, "TIER 6: Growth & Market Metrics"),
            (39, 43, "TIER 7: Trading & Investment Banking"),
            (44, 48, "TIER 8: Regulatory & Capital Management"),
            (49, 50, "TIER 9: Additional Balance Sheet Items")
        ]
        
        for start_idx, end_idx, tier_name in tier_ranges:
            print(f"\n{tier_name}:")
            tier_metrics = extractor.target_metrics[start_idx-1:end_idx]
            for i, metric in enumerate(tier_metrics, start_idx):
                availability = df[metric].notna().sum()
                percentage = (availability / len(df)) * 100 if len(df) > 0 else 0
                print(f"  {i:2d}. {metric:<30} ({availability:3d} records, {percentage:5.1f}%)")
        
    else:
        print("No data was extracted. Please check the logs for errors.")
        print("\nTroubleshooting suggestions:")
        print("1. Verify your User-Agent contains valid company name and email")
        print("2. Check your internet connection")
        print("3. Ensure the date range contains valid reporting periods")
        print("4. Review the logs above for specific error messages")

if __name__ == "__main__":
    main()

2025-06-30 19:20:03,468 - INFO - Starting data download for 2 banks from 2019-01-01 to 2025-12-31
2025-06-30 19:20:03,469 - INFO - Extracting 52 financial metrics
2025-06-30 19:20:03,470 - INFO - Processing 1/2: JPMorgan Chase & Co.


SEC Bank Earnings Data Extractor - Enhanced Version
Target banks: 2
Target metrics: 52 (prioritized by stakeholder importance)

Metric Priority Tiers:
TIER 1 (1-7): Core Profitability & Performance
TIER 2 (8-15): Capital & Balance Sheet Strength
TIER 3 (16-21): Risk Management & Credit Quality
TIER 4 (22-28): Income Statement Detail
TIER 5 (29-33): Operational Efficiency
TIER 6 (34-38): Growth & Market Metrics
TIER 7 (39-43): Trading & Investment Banking
TIER 8 (44-48): Regulatory & Capital Management
TIER 9 (49-50): Additional Balance Sheet Items



2025-06-30 19:20:48,033 - INFO - Extracted 25 records for JPMorgan Chase & Co.
2025-06-30 19:20:48,136 - INFO - Processing 2/2: Bank of America Corp
2025-06-30 19:21:32,187 - INFO - Extracted 25 records for Bank of America Corp
2025-06-30 19:21:32,368 - INFO - Calculated derived financial metrics
2025-06-30 19:21:32,383 - INFO - Successfully extracted 50 total records with 52 metrics



Data extraction completed!
Total records: 50
Companies with data: 2
Date range: 2019-03-31 00:00:00 to 2025-03-31 00:00:00


2025-06-30 19:21:33,235 - INFO - Data saved to bank_earnings_data_2019-01-01_2025-12-31.xlsx
2025-06-30 19:21:33,238 - INFO - Excel file contains 3 sheets: Bank_Earnings_Data, Summary, and Metrics_Dictionary



Data saved to: bank_earnings_data_2019-01-01_2025-12-31.xlsx

Data Quality Summary:
Data completeness: 41.1%
Total possible data points: 2,600
Actual data points extracted: 1,069

Top 10 metrics by data availability:
 1. NetIncome                   50 records (100.0%)
 2. NetInterestIncome           50 records (100.0%)
 3. ReturnOnEquity              50 records (100.0%)
 4. ReturnOnAssets              50 records (100.0%)
 5. ShareholdersEquity          50 records (100.0%)
 6. EarningsPerShare            50 records (100.0%)
 7. EarningsPerShareDiluted     50 records (100.0%)
 8. TotalAssets                 50 records (100.0%)
 9. TotalDeposits               50 records (100.0%)
10. NonInterestIncome           50 records (100.0%)

Sample data preview (Top 10 priority metrics):
  Datetime                 CompanyName   NetIncome  TotalRevenue  NetInterestIncome ReturnOnEquity ReturnOnAssets  EarningsPerShare  EarningsPerShareDiluted   TotalAssets  ShareholdersEquity  TotalDeposits
2019-03-